In [ ]:
import requests 
import inquirer
import pandas as pd
import numpy as np
from config import riot_api

In [ ]:
# region code is [{BR1:"Brazil",EUN1:"EU Nordic & East",EUW1:"EU West",JP1:"Japan",KR:"Korea",LA1:"Latin America North" 
# LA2:"Latin America South,NA1:"North America,OC1:"Oceania",PBE1:"Public Beta Environment,RU:"Russia",TR1:Turkey}]

region_code = input("Enter Your Region: ").upper() # Look up by Region

# Free Champions Rotation

url = "https://na1.api.riotgames.com/lol/platform/v3/champion-rotations?api_key=" + riot_api
champ_list = requests.get(url).json()

# Live Games

url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/featured-games?api_key=" + riot_api
live_games = requests.get(url)
if live_games.status_code == "200":
    live_games_status = "Region availible"
else:
    live_games_status ="Region unavailible"

live_games = live_games.json()
print(f"{live_games_status}\n{live_games['status']['status_code']}")


In [ ]:
name = input("Enter Your Summoner Name: ") # And Username

In [ ]:
# Summoners Data 

def Summoner (region_code, name):
    if region_code in ["NA", "BR", "LAN", "LAS", "OCE"]:
        region = "americas"
    elif region_code in ["KR", "JP"]:
        region = "asia"
    else:
        region = "europe"
    url = f"https://{region_code}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{name}?api_key=" + riot_api
    summoner_info_1 = requests.get(url).json()
    iden = summoner_info_1["id"]
    puuid = summoner_info_1["puuid"]
    url = f"https://{region_code}.api.riotgames.com/lol/league/v4/entries/by-summoner/{iden}?api_key=" + riot_api
    summoner_info_2 = requests.get(url).json()
    url = f"https://{region_code}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{iden}?api_key=" + riot_api
    summoner_info_3 = requests.get(url).json()
    
    url = f"https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=20&api_key=" + riot_api
    summoner_info_4 = requests.get(url).json()
    summoner_info_5 = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}?api_key=" + riot_api).json()) 
                       for match in summoner_info_4]
    summoner_info_6 = [(requests.get(f"https://{region}.api.riotgames.com/lol/match/v5/matches/{match}/timeline?api_key=" + riot_api).json()) 
                       for match in summoner_info_4]
    game_info = list(zip(summoner_info_4, summoner_info_5, summoner_info_6))
    
    if live_games_status == "Region availible":
        url = f"https://{region_code}.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{iden}?api_key=" + riot_api
        in_game_info =  requests.get(url)
        if in_game_info.status_code == "200":
            in_game_info =  in_game_info.json()
            print("In Live Game")
        else:
            in_game_info = np.nan
            print("No Live Data")
    else:
        in_game_info = np.nan
    return [summoner_info_1, summoner_info_2, summoner_info_3, game_info, in_game_info]

summoner_info = Summoner(region_code, name)



In [ ]:
Last_20_Games_Metadata = summoner_info[3]
Champ_Pool_Mastery = summoner_info[2]
Ranked_info = summoner_info[1]

summoner_info[0]

In [ ]:
Last_20_Games_df = pd.DataFrame(Last_20_Games_Metadata)
Last_20_Games_df

In [ ]:
# Ranked Data

queue = input("Enter Your Queue Type: ") # Enter Queue Type [RANKED_SOLO_5x5, RANKED_TFT,
                                        # RANKED_FLEX_SR, RANKED_FLEX_TT]
tier = input("Enter Your Tier: ") # Enter Tier Level [IRON, BRONZE, SILVER, GOLD, 
                                        # PLATINUM, DIAMOND, MASTER, GRANDMASTER, CHALLENGER]
division = input("Enter Your Division: ") # Enter Division Level [IV, III, II, I]
page = 1 #input("Page Number: ")
url = f"https://na1.api.riotgames.com/lol/league-exp/v4/entries/{queue}/{tier}/{division}?page={page}&api_key=" + riot_api
rank_info = requests.get(url).json()

rank_info